In [77]:
import matplotlib
matplotlib.use('Agg')
%pylab inline

import os
import math
import pandas
import datetime as dt
from time import time
import numpy as np
import pandas as pd

from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.util.common import *
from bigdl.dataset.transformer import *

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from pyspark import SparkContext

sc.stop
sc=SparkContext.getOrCreate(conf=create_spark_conf().setMaster("local[4]").set("spark.driver.memory","64g"))

init_engine()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/python2.7/dist-packag

Populating the interactive namespace from numpy and matplotlib


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:45754)

In [78]:
import os

cwd = os.getcwd()
print cwd

os.chdir("/notebooks")


cwd = os.getcwd()
print cwd
print os.listdir(os.path.join(cwd, 'data'))

/notebooks
/notebooks
['muchdata-100-100-40_old.npy', 'stage1', 'stage1_labels.csv.zip', 'stage1_sample_submission.csv', 'stage1.7z', 'muchdata-50-50-20_oldV2.npy', 'muchdata-50-50-20_old.npy', 'muchdata-100-100-40.npy', 'stage1_sample_submission.csv.zip', 'stage1_labels.csv', 'muchdata-50-50-20_orig.npy', 'muchdata-50-50-20.npy', 'muchdata-50-50-20_t2.npy', 'muchdata-32-32-32.npy']


In [79]:
import numpy as np
import time

In [80]:
much_data = np.load('./data/muchdata-32-32-32.npy', encoding = 'latin1')

In [81]:
print len(much_data)
print len(much_data[0][0])
print len(much_data[0][1])

1397
32
2


In [82]:
train_data = much_data[:16]
validation_data = much_data[16:24]

In [86]:
# creat the RDD for model training now
prep_train_rdd = sc.parallelize(train_data,4)
prep_test_rdd = sc.parallelize(validation_data,4)

print prep_train_rdd.take(1)[0][0].transpose().shape

train_rdd = prep_train_rdd.map(lambda s : Sample.from_ndarray(np.array(s[0]).reshape(1,32,32,32), np.array(s[1][1]+1)))
test_rdd = prep_test_rdd.map(lambda s : Sample.from_ndarray(np.array(s[0]).reshape(1,32,32,32), np.array(s[1][1]+1)))

OSError: [Errno 2] No such file or directory: '/tmp/spark-cbe80c60-1d63-445e-a978-d5859f73f19e/pyspark-b37e718e-dcab-43ab-aaec-8875f5022161/tmpCfnRsv'

In [56]:
train_rdd.take(1)

[Sample: features: [JTensor: storage: [-2000. -2000. -2000. ... -2000. -2000. -2000.], shape: [ 1 32 32 32], float], labels: [JTensor: storage: [2.], shape: [1], float]]

In [ ]:
# Create a model simpler than LeNet model to test the data pipeline and layer setup
# here the input number to the full connected Linear layer is
# based on the 3D Max pool
# 6*(32/2)*(32/2)*(32/2) = 24576

def build_model(class_num):
    model = Sequential()
    model.add(Reshape([1,32,32,32]))
    model.add(VolumetricConvolution(3, 6, 5, 5, 5).set_name('3Dconv1'))
    model.add(Tanh())
    model.add(Echo())
    model.add(VolumetricMaxPooling(2, 2, 2, \
                                   1, 1, 1, \
                                   0, 0, 0).set_name('3Dpool1'))
    model.add(Echo())
    model.add(Reshape([6 * 16 * 16 * 16]))
    model.add(Linear(6 * 16 * 16 * 16, 128).set_name('fc1'))
    model.add(Tanh())
    model.add(Echo())
    model.add(Linear(128, class_num).set_name('score'))
    model.add(LogSoftMax())
    return model
lenet_model = build_model(2)

In [57]:
# Create a model simpler than LeNet model to test the data pipeline

def build_model(class_num):
    model = Sequential()
    model.add(Reshape([1,32,32,32]))
    model.add(VolumetricConvolution(3, 6, 5, 5, 5).set_name('3Dconv1'))
    model.add(Tanh())
    model.add(VolumetricMaxPooling(2, 2, 2, \
                                   1, 1, 1, \
                                   0, 0, 0).set_name('3Dpool1'))
    model.add(Reshape([81 *18 *18 * 18]))
    model.add(Linear(81 * 18 * 18 * 18, 16).set_name('fc1'))
    model.add(Tanh())
    model.add(Linear(16, class_num).set_name('score'))
    model.add(LogSoftMax())
    return model
lenet_model = build_model(2)

creating: createSequential
creating: createReshape
creating: createVolumetricConvolution
creating: createTanh
creating: createVolumetricMaxPooling
creating: createReshape
creating: createLinear
creating: createTanh
creating: createLinear
creating: createLogSoftMax


In [64]:
# Create an Optimizer

optimizer = Optimizer(
    model=lenet_model,
    training_rdd=train_rdd,
    criterion=ClassNLLCriterion(),
    optim_method=SGD(learningrate=0.4, learningrate_decay=0.0002),
    end_trigger=MaxEpoch(3),
    batch_size=8)

# Set the validation logic
optimizer.set_validation(
    batch_size=8,
    val_rdd=test_rdd,
    trigger=EveryEpoch(),
    val_method=[Top1Accuracy()]
)

app_name='lenet-'+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
train_summary = TrainSummary(log_dir='/tmp/bigdl_summaries',
                                     app_name=app_name)
train_summary.set_summary_trigger("Parameters", SeveralIteration(50))
val_summary = ValidationSummary(log_dir='/tmp/bigdl_summaries',
                                        app_name=app_name)
optimizer.set_train_summary(train_summary)
optimizer.set_val_summary(val_summary)
print "saving logs to ",app_name

creating: createClassNLLCriterion
creating: createDefault
creating: createSGD
creating: createMaxEpoch
creating: createDistriOptimizer
creating: createEveryEpoch
creating: createTop1Accuracy
creating: createTrainSummary
creating: createSeveralIteration
creating: createValidationSummary
saving logs to  lenet-20180207-151533


In [65]:
%%time
start = time()
# Boot training process
trained_model = optimizer.optimize()
print "Optimization Done in {} seconds.".format(time()-start)

Py4JJavaError: An error occurred while calling o1076.optimize.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 87.0 failed 1 times, most recent failure: Lost task 0.0 in stage 87.0 (TID 88, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1435)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1423)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1422)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1422)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:802)
	at org.a

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 47440)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python2.7/SocketServer.py", line 649, in __init__
    self.handle()
  File "/opt/work/spark-2.1.1/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/opt/work/spark-2.1.1/python/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError


In [ ]:
# original net: Create a LeNet model
"""
def build_model(class_num):
    model = Sequential()
    model.add(Reshape([1,32,32,32]))
    model.add(VolumetricConvolution(3, 6, 5, 5, 5).set_name('3Dconv1'))
    model.add(Tanh())
    model.add(VolumetricMaxPooling(2, 2, 2, \
                                   1, 1, 1, \
                                   0, 0, 0).set_name('3Dpool1'))
    model.add(Tanh())
    model.add(VolumetricConvolution(6, 12, 5, 5, 5).set_name('3Dconv2'))
    model.add(VolumetricMaxPooling(2, 2, 2, \
                                   1, 1, 1, \
                                   0, 0, 0).set_name('3Dpool2'))
    model.add(Reshape([12 * 22 * 22 * 22]))
    model.add(Linear(12 * 22 * 22 * 22, 16).set_name('fc1'))
    model.add(Tanh())
    model.add(Linear(16, class_num).set_name('score'))
    model.add(LogSoftMax())
    return model
lenet_model = build_model(2)
"""